<h2>Wprowadzenie</h2>

<h3>Zestawy danych</h3>

**Movies Dataset**

The Movies Dataset został pobrany z serwisu Kaggle https://www.kaggle.com/rounakbanik/the-movies-dataset.

Pliki zawierają metadane na temat 45 000 filmów pochodzących z oficjalnego zestawu danych MovieLens Dataset. Filmy pochodzą sprzed lipca 2017, zebrane są informacje takie jak obsada, zespół, fabularne słowa kluczowe, budżet, przychód, plakaty, daty wydania, języki wydania, firmy produkcyjne, kraje, liczba głosów TMDB i średnie głosów. Zestaw danych zawiera również pliki z 26 milionami ocen od 27 000 użytkownikówd dla wszystkich 45 000 filmów. Oceny wystawione zostały w skali od 1 do 5, pobrano je z oficjalnej strony MovieLens.

Analiza będzie przebiegać przy wykorzystaniu następujących plików:

- _moviesmetadata.csv: Główny plik metadanych filmów. Zawiera informacje o 45 000 filmach zawartych w zestawie danych Full MovieLens. Funkcje obejmują plakaty, tła, budżet, przychód, daty wydania, języki wydania, kraje produkcji i firmy produkcyjne.

- keywords.csv: Zawiera słowa kluczowe fabuły filmowej dla filmów MovieLens. Dostępny w postaci obiektu JSON.

- credits.csv: _Zawiera informacje o obsadzie i ekipie wszystkich naszych filmów. Dostępny w postaci obiektu JSON.

- links.csv: Zawiera identyfikatory TMDB i IMDB wszystkich filmów znajdujących się w zestawie danych Full MovieLens.

- _linkssmall.csv: Zawiera identyfikatory TMDB i IMDB małego podzbioru 9 000 filmów z pełnego zestawu danych.

- ratings.csv: Oceny filmów wystawione przez użytkowników

- _ratingssmall.csv: Podzbiór 100 000 ocen od 700 użytkowników do 9 000 filmów.

**MovieLens**

MovieLens został pobrany z https://grouplens.org/datasets/movielens/latest/

Ze względu na problemy wydajnościowe przy wykorzystaniu pełnego zestawu danych (27 753 444 ocen, 1 108 997 aplikacji tagów, 58 098 filmów, 283 228 użytkowników), wykorzystano ml-latest-small.zip. Zestaw danych zawiera 100 000 ocen i 3 600 aplikacji tagów w 9 000 filmach. Dane te zostały utworzone przez 600 użytkowników w okresie od 09 stycznia 1995 do 26 września 2018 roku. Oceny są w skali od 1 do 5.

Do utworzenia systemów rekomendacyjnych posłużą dwa pliki z zestawów danych MovieLens: ratings.csv oraz movies.csv. Dane ocen to oceny filmów wystawione przez użytkowników. W każdym wierszu znajdują się trzy pola:

- userId,
- movieId,
- rating

Każdy wiersz może być rozumiany jako zapis interakcji między użytkownikiem, a filmem. Dane filmów zawierają tytuł filmu i gatunki dla każdego identyfikatora filmu w danych ocen.

<h2>Prosty system rekomendacyjny</h2>

Najprostszą metodą przy tworzeniu rekomendacji jest zaproponowanie użytkownikowi najbardziej popularnych produktów (ang. Simple Recommender). 

Do załadowania zestawu danych wykorzystano bibiliotekę **pandas**. Biblioteka ta służy głównie do manipulacji i analizy danych. Dane zaprezentowane są w formie tabelarycznej. Biblioteka Pandas jest wspierana przez tablicę NumPy do implementacji obiektów danych. Oferowane są gotowe struktury danych i operacje do manipulowania tabelami liczbowymi, szeregami czasowymi, obrazami i zbiorami danych przetwarzania języka naturalnego. Więcej informacji o pandas można się dowiedzieć z oficjalnej dokumentacji https://pandas.pydata.org/. 

Fragment kodu przedstawiony poniżej to wgranie pliku _movies_metadata.csv_ do tablicy danych _metadata_. Przed przystąpieniem do analizy danych, należy również poznać szczegóły danych.

In [ ]:
pip install pandas
pip install scikit-learn
pip install numpy
pip install time
pip install scipy
pip install fuzzywuzzy

In [10]:
# https://www.datacamp.com/community/tutorials/recommender-systems-python
import pandas as pd

metadata = pd.read_csv('data/MoviesDataset/movies_metadata.csv', low_memory=False)
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Dodatkowo warto zapoznać się z podstawowymi statystykami opisowymi zestawu danych, takimi jak centralna tendencja, rozproszenie i kształt rozkładu zbioru. Metoda **describe** służy do przeglądania podstawowych szczegółów statystycznych, takich jak percentyl, średnia czy odchylenie standardowe ramki danych lub serii wartości liczbowych.

In [ ]:
metadata.describe()

Celem systemu rekomendacyjnego w tej sekcji jest odnalezienie najbardziej popularnych filmów, które z dużym prawdopodobieństwem będą podobać się użytkownikom systemu. Jednakże, działanie polegające na posortowaniu filmów zgodnie z wystawionymi opiniami, może doprowadzić do zaburzonych wyników.

Przy takiej implementacji metody, system nie brałby pod uwagę popularności filmu. W takiej sytuacji film ze średnią oceną 9, wystawioną przez 10 głosujących zostanie uznany za lepszy, niż film z oceną 8,9 na 10 000 głosów. W związku z tym wskaźnik faworyzowałby filmy z mniejszą liczbą głosujących z bardzo wysokimi ocenami. Wraz ze wzrostem liczby głosujących, ocena filmu jest regulowana i zbliża się do wartości, która odzwierciedla jakość filmu i daje użytkownikowi znacznie lepsze wyobrażenie o tym, który film powinien wybrać. Czasem dla filmów z bardzo małą liczbą wyborców, należy wziąć pod uwagę dodatkowe czynniki zewnętrzne.

Biorąc pod uwagę potencjalne problemy opisane powyżej, zaleca się wykorzystanie oceny ważonej, która uwzględnia średnią ocenę i liczbę zgromadzonych głosów. Taki system sprawi, że film z 9 oceną na 100 000 głosujących uzyska znacznie wyższą punktację, niż film o tej samej ocenie, ale zaledwie od kilkuset głosujących.

IMDB to internetowa baza danych na temat filmów i ludzi z nim związanych. Zawiera informacje o aktorach, reżyserach, scenarzystach, producentach, montażystach, operatorach, muzykach. W poniższej implementacji systemu wykorzystano ważoną formułę oceny dla Top 250 IMBD, przedstawioną za pomocą następującej formuły matematycznej.

<br>\begin{equation}
    WeightedRating(WR) = (\frac{v}{v+m}*R) + (\frac{m}{v+m}*C)
\end{equation}

gdzie:

- $v$ to liczba głosów na dany film
- $m$ to minimalna liczba głosów, wymagana aby film został uwzględniony
- $R$ to średnia ocena danego filmu
- $C$ to średnia ocena filmów udzielona przez wszystich użytkowników

W zestawie danych istnieje informacja o liczbie głosów na dany film (_vote_count_) oraz uzyskanej średniej ocenie (_vote_average_). Na podstawie tych danych można obliczyć średnią ocenę filmów udzieloną przez wszystich użytkowników. Dodatkowo, przy użyciu kwantylu, w zbiorze danych zostaną jedynie filmy o znaczącej ilości głosów. Kwantyle to wartości cechy badanej zbiorowości, które dzielą uporządkowaną zbiorowość statystyczną na określone, równe części pod względem liczby jednostek statystycznych. Kwantylem rzędu $ q, gdzie (0<q<1) $ nazywamy taką liczbę $Q$, że $ q * 100\% $ elementów danej zbiorowości statystycznej ma wartość nie większą od $Q$. 

Poniższa implementacja przyjmuje wartość $q$ na poziomie 90%, co oznacza, że do tabeli danych _movies_ zostaną przypisane filmy, które uzyskały więcej głosów niż 90% obiektów w zestawie danych.

In [14]:
C = metadata['vote_average'].mean()
m = metadata['vote_count'].quantile(0.90)

movies = metadata.copy().loc[metadata['vote_count'] >= m]

Następnym krokiem jest zdefiniowanie funkcji, która zwracać będzie średnią ważoną ocen dla filmu. Parametry, przekazywane do funkcji to lista filmów, średnia wszystich głosów oraz mininamalna liczba głosów, potrzebna do uwzględnienia filmu w analizie. Obliczenia bazują na opisanej wcześniej formule matematycznej dla Top 250 IMBD. Oznaczenia we wzorze również pozostają takie same.

In [ ]:
def weighted_rating(movies, m=m, C=C):
    v = movies['vote_count']
    R = movies['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

Do tabeli danych _movies_ zostanie dodana nowa kolumna, której wartości obliczane są za pomocą powyższej metody _weighted_rating_. W celu uzyskania finalnych propozycji najpopularniejszych filmów, należy posortować tabelę danych w kolejności malejącej na podstawie kolumny z wartościami oceny ważonej. Do reprezentacji wyników wypisano tytuł, liczbę głosów, średnią ocenę oraz ważoną ocenę 20 najlepszych filmów.

In [ ]:
movies['score'] = movies.apply(weighted_rating, axis=1,)

movies = movies.sort_values('score', ascending=False)
movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

Porównanie uzyskanych wyników z listą Top 250 IMDB (https://www.imdb.com/chart/top/) wskazuje na wiele podobnych filmów, takich jak „Skazani na Shawshank” i „Ojciec chrzestny”.

<h2>System oparty o filtrowanie kontekstowe (Tłumaczenie?)</h2>

Obserwując zachowania i tendencje towarzyszące podejmowaniu decyzji, można zauważyć, ̇ze ludzie mają skłonność do wyboru produktów podobnych do tych, które podobały im się w przeszłości. Na tym przekonaniu bazuje idea systemów opartych o filtrowanie kontekstowe (ang. Content-based Filtering), gdzie polecony zostaje artykuł najbardziej zbliżony do poprzednio polubionych przez użytkownika. 

W tej sekcji zostanie przedstawiona implementacja sytemu rekomendacyjnego, opartego o filtrowanie kontekstowe. Polega na uwzględnieniu słów kluczowych z opisu produktu, bądź wystawionej przez użytkownika oceny w celu stworzenia wektorów w przestrzeni cech. Podobieństwo między wektorami preferencji użytkownika oraz atrybutami produktu obliczenie jest za pomocą miar podobieństwa, takich jak podobieństwo kosinusowe.

Do załadowania zestawu danych wykorzystano bibiliotekę **pandas**. Fragment kodu przedstawiony poniżej to wgranie pliku _movies_metadata.csv_ do tablicy danych _metadata_. Ze względu na problemy wydajnościowe podczas analizy podad 45 000 filmów, tablica danych została zmniejszona do 25 000 rekordów.

In [ ]:
# https://www.datacamp.com/community/tutorials/recommender-systems-python
import pandas as pd

metadata = pd.read_csv('data/MoviesDataset/movies_metadata.csv', low_memory=False)
metadata = metadata[:25000]

Słowa kluczowe, charakteryzujące cechy produktu mogą być dostarczone przy dodawaniu artykułu do serwisu. Nie jest to jednak obowiązkowe działanie, dlatego brakujące atrybuty mogą zostać odnalezione w opisie produktu lub opinii wystawionych na jego temat. 

Jest to problem przetwarzania języka naturalnego. Należy wyodrębnić pewne cechy z danych tekstowych, obliczyć wektory cech dla każdego filmu, a następnie obliczyć podobieństwo między nimi. Wektory cech to wektorowa reprezentacja słów kluczowych, która niesie za sobą znaczenie semantyczne. Atrybuty filmu będą wyszukiwane w kolumnie _overview_ tabeli danych, przedstawionej poniżej.

In [ ]:
metadata['overview'].head(10)

<h3>Wyodrębnienie słów kluczowych z opisu filmu</h3>

Zarówno model preferencji, jak i wiedza na temat produktów zorientowana jest na atrybuty istniejących w systemie artykułów. Cechy artykułu mogą zostać pozyskane przy wykorzystaniu przetwarzania języka naturalnego, a konkretnie metody TF-IDF (ang. Term Frequency and Inverse Document Frequency), pozwalającej na reprezentację cech w przestrzeni wektorowej. Jest to jedna z metod obliczania wagi słów w oparciu o liczbę ich wystąpień, należąca do grupy algorytmów obliczających statystyczne wagi termów. TF-IDF informuje o częstości wystąpienia termów uwzględniając jednocześnie odpowiednie wagi znaczenia lokalnego termu i jego znaczenia w kontekście pełnej kolekcji.

Wynikiem, uzyskanym po zastosowaniu TF-IDF jest macierz, w której każda kolumna reprezentuje słowo ze słownika ogólnego (wszystkie słowa, które pojawiają się w co najmniej jednym dokumencie). Rezultat metody TF-IDF to częstotliwość występowania słowa w opisie filmu, pomniejszona o liczbę opisów, w których występuje. Ma to na celu zmniejszenie znaczenia słów, które często pojawiają się w przeglądach fabuły, a tym samym ich znaczenia przy obliczaniu końcowego wyniku podobieństwa.

Do wyodrębnienia słów kluczowych zastosowana zostanie moduł Scikit-learn. Jest to darmową biblioteką algorytmów z dziedziny uczenia maszynowego, napisana w języku Python i zbudowaną na bazie modułu SciPy. Moduł Scikit-learn udostępnia wiele algorytmów z dziedziny nadzorowanego i nienadzorowanego uczenia maszynowego w postaci spójnego interfejsu programistycznego. Scikit-learn posiada wbudowaną klasę TfIdfVectorizer, która tworzy macierz TF-IDF. Po zaimportowaniu modułu TF-IDF, należy usunąć z opisu słowa, które nie dostarczają żadnej wartości do analizy, takie jak "the" czy "an".

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
metadata['overview'] = metadata['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

ModuleNotFoundError: No module named 'sklearn'

Za pomocą metody _shape_ zobaczyć można jak wiele słów kluczowych zostało wyodrębionych. Wynik (25000, 53130) informuje o odnalezieniu 53 130 odrębnych słów dla 25 000 filmów. Dodatkowo, za pomocą funkcji _get_feature_names_, sprawdzić można jakie słowa zostały wydrębione. Czasem nie przynoszą one bezpośredniej wartości dla użytkownika, jednak dla systemu częstotliwość ich występowania pomaga zdecydować o charakterystyce filmu.

In [ ]:
print(tfidf_matrix.shape)
print(tfidf.get_feature_names()[25000:25010])

<h3>Obliczanie miary podobieństwa wektorów</h3>

Mając do dyspozycji macierz tf-idf, można obliczyć podobieństwo między wektorami cech, w celu odnalezienia zbliżonych do siebie filmów. Zidentyfikowanie podobnych do siebie wektorów, pozwala na trafną rekomendację produktu. Miarę podobieństwa między wektorami można obliczyć, używając wybranego sposobu:

- odległość Euklidesowa
- współczynnik korelacji Pearsona
- odległość kosinusowa

Nie istnieje konkretna odpowiedź na pytanie, który miara podobieństwa jest najlepsza. Różne podejścia sprawdzają się dobrze w rozmaitych scenariuszach, dlatego też dobrym pomysłem jest eksperymentowanie z różnymi danymi i obserwowanie wyników.

Wynik podobieństwa kosinusowego, jest niezależny od wielkości danych oraz stosunkowo łatwy i szybki do obliczenia. Miara kosinusowa jest szczególnie efektywna, jeżeli obliczenia wykonywane są na wynikach TF-IDF. Matematycznie definiowana jest w następujący sposób:

<br>\begin{equation}
    \cos(\phi) = \frac{X\cdot Y}{|X|\hspace{0.05cm}|Y|} = \frac{\sum_{i=1}^n x_i y_i}{\sqrt{\sum_{i=1}^n x_i^2}\sqrt{\sum_{i=1}^n y_i^2}}
\end{equation}

Zastosowanie wektoryzatora TF-IDF, sprawia że obliczenie iloczynu skalarnego między każdym wektorem bezpośrednio to wynik podobieństwa kosinusowego. Z tego powodu, do obliczenia podobieństwa wektorów, użycie metody _linear_kernel()_ biblioteki _sklearn_ jest bardziej efektywne, niż klasyczna metoda _cosine_similarities()_.

Dodatkowo, do prawidłowego funkcjonowania systemu rekomendacyjnego należy zastosować mapowanie filmów i indeksów tabeli danych. Pozwoli to na późniejszą identyfikację indeksu, biorąc pod uwagę tytuł filmu.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

_Cosine_sim_ to macierz o rozmiarze (25000, 25000), co oznacza, że zawiera wyniki podobieństwa kosinusowego każdego filmu z każdym innym w zestawie danych. Wyświetlenie danych dla jednego filmu to wektor kolumnowy o rozmiarze (1,25000), gdzie kolumna zawiera wartości podobieństwa z każdym filmem.

In [ ]:
print(cosine_sim.shape)
print(cosine_sim[1])

<h3>Rekomendacje filmów</h3>

Poniżej przedstawiono implementację głównej funkcji systemu, która zwraca listę filmów podobnych do tytułu w parametrach metody. Oprócz tego, metoda wymaga również przekazania macierzy z obliczonymi podobieństwami wektorów cech. Algorytm odnalezienie najlepszych propozycji przebiega w następujących krokach:

- Odnalezienie indeksu dla filmu, którego tytuł został podany w parametrach funkcji
- Uzyskanie wyników podobieństwa kosinusowego dla podanego filmu z pozostałymi w macierzy
- Przeksztacenie wyników w listę krotek, gdzie pierwszy element to indeks filmu, a drugi to wynik podobieństwa
- Posortowanie wyżej wymienionej listy krotek na podstawie wyników podobieństwa
- Wydzielenie 10 najbliższych filmów z tej listy, z pominięciem pierwszego elementu
- Zwrócenie tytułów, odpowiadających indeksom elementów

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return metadata['title'].iloc[movie_indices]

Wywołanie metody _get_recommendations_ wraz z wybranym tytułem filmu zwraca listę tytułów, najbardziej zbliżonych do filmu przekazanego w parametrach. Przykładowo dla filmu "Mroczny rycerz powstaje" (ang. The Dark Knight Rises) w reżyserii Christophera Nolana, zwrócone zostać powinny filmy akcji o podobnej tematyce. 

In [ ]:
get_recommendations('The Dark Knight Rises')

<br>System rekomendacyjny bardzo trafnie przewidział, które filmy są podobne do "Mroczny rycerz powstaje". Zauważyć można, że wszystkie polecone pozycje to seria filmów Batmana, co zdecydowanie ma swoje zastosowanie biznesowe. Jednak polecanie filmów tak bardzo zbliżonych do siebie sprawia, że jakość rekomendacji spada. Użytkownicy systemu, którym podobał się film "Mroczny rycerz powstaje" mogą być zainteresowani innymi pozycjami filmowymi reżyserii Christophera Nolana. Jest to zatem czynnik, którego aktualny system rekomendacyjny nie może uchwycić.

<h3>Uwzględnienie szczegółowych metadanych filmu</h3> 

Jakość systemu rekomendacyjnego może zostać zwiększona poprzez uwzględnienie bardziej szczegółowych metadanych. W poprzedniej części cechy filmu wyodrębnione zostały z opisu, co przyczyniło się do zaproponowania jedynie tych filmów, których fabuła jest zbliżona do tytułu, przekazanego w parametrze funkcji. 

W tej części notatnika przedstawione zostanie jak zaimplementować system rekomendacyjny w oparciu o następujące metadane: 
- reżyser filmu
- 3 najlepszych aktorów
- pokrewne gatunki
- słowa kluczowe fabuły filmu

Słowa kluczowe, obsada i dane zespołu nie są dostępne w bieżącym zbiorze danych. Pierwszym krokiem jest więc pobranie i dołączenie ich do głównych danych z pliku _movies_metadata.csv_.

In [6]:
import numpy as np
import pandas as pd

credits = pd.read_csv('data/MoviesDataset/credits.csv')
keywords = pd.read_csv('data/MoviesDataset/keywords.csv')
metadata = pd.read_csv('data/MoviesDataset/movies_metadata.csv', low_memory=False)
metadata = metadata[:20000]

metadata = metadata.drop(19730)

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


<h3>Przygotowanie metadanych</h3>

Aby uzyskać metadane, potrzebne do tej części implementacji należy wyodrębić najważniejszych aktorów, reżysera i słowa kluczowe związane z filmem. Utworzenie nowych kolumn w tablicy danych wymaga ich przekonwertowania do użytecznego formatu. Zastosowanie _literal_eval_ pozwala odnaleźć typ wartości przechowywanej w pliku lub tekstowym typie danych. Po konwersji można łatwo tworzyć i manipulować strukturami danych podczas wykonywania programua, a także zapisać je w plikach.

In [13]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

Następnym krokiem jest utworzenie funkcji, które zwracać będą potrzebne metadane. 

Metoda _get_director_ zwraca imię reżysera, dla przekazanego w parametrze obiektu danych, dotyczącego zespołu pracującego przy filmie. 

Zadaniem metody _get_list_ jest wyodrębnienie trzech pierwszych elementów lub całej listy, w zależności od jej długości. Funkcja ta zostanie wykorzystana przy metadanych, takich jak obsada, słowa kluczowe i gatunki filmowe.

In [14]:
def get_director(crew):
    for member in crew:
        if member['job'] == 'Director':
            return member['name']
    return np.nan


def get_list(data):
    if isinstance(data, list):
        names = [el['name'] for el in data]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [15]:
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

Poniżej wyświetlono wyodrębnione metadane:

- obsada (ang. cast)
- reżyser (ang. director)
- słowa kluczowe (ang. keywords)
- gatunki filmowe (ang. genres)

In [16]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[Comedy]


<h3>Przetwarzanie wstępne</h3>

Ważnym etapem przetwarzania wstępnego danych jest przekonwertowanie metadanych na małe litery i usunięcie wszystkich spacji między nimi. Działanie to jest konieczne do prawidłowego działania wektoryzatora. Przykładowo dla aktorów "Johnny Depp" oraz "Johnny Galecki" metoda uwzględniłaby jedynie imię, a więc zostaliby oni potraktowani jako ta sama osoba. Po etapie przetwarzania wstępnego wyżej wymienieni aktorzy będą reprezentowani jako „johnnydepp” i „johnnygalecki”, co pozwoli na ich odróżnienie.

In [17]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [18]:
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

Poniżej wyświetlono metadane po zmianie liter na małe oraz usunięciu spacji:

In [19]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[tomhanks, timallen, donrickles]",johnlasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[boardgame, disappearance, basedonchildren'sbook]","[adventure, fantasy, family]"
2,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"[fishing, bestfriend, duringcreditsstinger]","[romance, comedy]"
3,Waiting to Exhale,"[whitneyhouston, angelabassett, lorettadevine]",forestwhitaker,"[basedonnovel, interracialrelationship, single...","[comedy, drama, romance]"
4,Father of the Bride Part II,"[stevemartin, dianekeaton, martinshort]",charlesshyer,"[baby, midlifecrisis, confidence]",[comedy]


Należy utworzyć nową kolumnę w tablicy danych, która stanowić będzie ciąg, zawierającym wszystkie metadane, przekazywane do wektoryzatora. Poniższa funkcja _create_soup_ zwróci połączone wymagane kolumny (aktorzy, reżyser i słowa kluczowe) za pomocą spacji. Jest to ostatni etap przetwarzania wstępnego, a dane wyjściowe tej metody zostaną wprowadzone do modelu wektora słowa.

In [20]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [21]:
metadata['soup'] = metadata.apply(create_soup, axis=1)

Poniżej wyświetlono metadane po zmianie liter na małe oraz usunięciu spacji:

In [22]:
metadata[['soup']].head()

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...
2,fishing bestfriend duringcreditsstinger walter...
3,basedonnovel interracialrelationship singlemot...
4,baby midlifecrisis confidence stevemartin dian...


Kolejne kroki są takie same, jak w przypadku systemu rekomendacyjnego zaimplementowanego w sekcji powyżej, czyli opartego na opisie fabuły. 

Kluczową różnicą jest zastosowanie _CountVectorizer_ z biblioteki sklearn, zamiast TF-IDF. Głównym powodem jest brak komponentu odwrotnej częstotliwości dokumentu (IDF) w metodzie _CountVectorizer_. W obecnej sytuacji nie ma potrzeby, aby waga aktora lub reżysera zmiejszała się, jeśli zaistniał w stosunkowo większej liczbie filmów. 

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

ModuleNotFoundError: No module named 'sklearn'

Za pomocą metody _shape_ zobaczyć można jak wiele słów kluczowych zostało wyodrębionych. Wynik (20121, 36687) informuje o odnalezieniu 36 687 odrębnych słów. Dodatkowo, za pomocą funkcji _get_feature_names_, sprawdzić można jakie słowa zostały wydrębione. Czasem nie przynoszą one bezpośredniej wartości dla użytkownika, jednak dla systemu częstotliwość ich występowania pomaga zdecydować o charakterystyce filmu.

In [ ]:
print(count_matrix.shape)
print(count.get_feature_names()[25000:25010])

<h3>Obliczanie miary podobieństwa wektorów</h3>

Dodatkowo, do prawidłowego funkcjonowania systemu rekomendacyjnego należy zastosować mapowanie filmów i indeksów tabeli danych. Pozwoli to na późniejszą identyfikację indeksu, biorąc pod uwagę tytuł filmu.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

<h3>Rekomendacje filmów</h3>

Poniżej przedstawiono implementację głównej funkcji systemu, która zwraca listę filmów podobnych do tytułu w parametrach metody. Oprócz tego, metoda wymaga również przekazania macierzy z obliczonymi podobieństwami wektorów cech. Algorytm odnalezienie najlepszych propozycji przebiega w następujących krokach:

- Odnalezienie indeksu dla filmu, którego tytuł został podany w parametrach funkcji
- Uzyskanie wyników podobieństwa kosinusowego dla podanego filmu z pozostałymi w macierzy
- Przeksztacenie wyników w listę krotek, gdzie pierwszy element to indeks filmu, a drugi to wynik podobieństwa
- Posortowanie wyżej wymienionej listy krotek na podstawie wyników podobieństwa
- Wydzielenie 10 najbliższych filmów z tej listy, z pominięciem pierwszego elementu
- Zwrócenie tytułów, odpowiadających indeksom elementów

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return metadata['title'].iloc[movie_indices]

Wywołanie metody _get_recommendations_ wraz z wybranym tytułem filmu zwraca listę tytułów, najbardziej zbliżonych do filmu przekazanego w parametrach. Przykładowo dla filmu "Mroczny rycerz powstaje" (ang. The Dark Knight Rises) w reżyserii Christophera Nolana, zwrócone zostać powinny filmy akcji nie tylko o podobnej tematyce, lecz także uwzględniające metadane, takie jak reżyser, gatunek filmowy oraz obsada filmu.

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim)

<h2>Filtrowanie kolaboracyjne</h2>
    
W zaawansowanych systemach rekomendacyjnych bardzo ważne jest zagwarantowanie różnorodności polecanych produktów. Filtrowanie kolaboracyjne (ang. Collaborative Filtering) to technika używana w systemach rekomendacyjnych, która bazuje na idei, że podobni użytkownicy będą zainteresowani tymi samymi produktami. Tego rodzaju filtrowanie nie wymaga szczegółowych informacji na temat atrybutów produktu. Metody kolaboracyjne w systemach rekomendacyjnych bazują na interakcji między użytkownikami i produktami w systemie, co pozwala na utworzenie tzw. macierzy interakcji (ang. User-Item Interactions Matrix). 

W tej sekcji zostanie przedstawiona implementacja sytemu rekomendacyjnego, opartego o filtrowanie kolaboracyjne. Polega na wykorzystaniu działania użytkowników do polecania innych filmów. Popularnym podejściem do określenia podobieństwa użytkowników, wykorzystanym w tej sekcji, jest metoda **K najbliższych sąsiadów** (ang. K-nearest Neighbors Algorithm), pozwalająca na ich klasyfikację do odpowiadających profili preferencji. 

Do załadowania zestawu danych wykorzystano bibiliotekę **pandas**. Fragment kodu przedstawiony poniżej to wgranie plików _movies.csv_ oraz _ratings.csv_ do odpowiadających im tablic danych.

In [ ]:
# https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea

import time
import pandas as pd

movies = pd.read_csv('data/MovieLens/movies.csv', usecols=['movieId', 'title'], dtype={'movieId': 'int32', 'title': 'str'})
ratings = pd.read_csv('data/MovieLens/ratings.csv', usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
movies.head()

In [ ]:
ratings.head()

<h3>Określanie podobieństwa wektorów</h3>

Do implementacji algorytmu, opartego o filtrowanie kolaboracyjne oparte na obiekcie rekomendacji, metoda K najbliższych sąsiadów (KNN) jest jednym z lepszych podejść. Jest to jeden z algorytmów regresji nieparametrycznej, używanych w statystyce do prognozowania wartości pewnej zmiennej losowej oraz do klasyfikacji. 

KNN opiera się na podobieństwie cech elementów. na podstawie obliczonej „odległości” czyli różnicy między filmem docelowym, a każdym innym filmem w bazie danych. Posortowanie wartości pozwala na odnalezienie najbardziej zbliżonych wektorów cech, opisujących filmy, jako najlepsze rekomendacje.

Algorytm odnajdywania najbliższych sąsiadów składa się z poniższych kroków:

1. Wybranie $k$ punktów centralnych.
2. Obliczenie odległości pozostałych punktów do punktów centralnych. Przypisanie punktów do najbliżej zlokalizowanych środków klastrów.
3. Obliczenie średniej wartości elementów znajdujących się w klastrze i nadpisanie wartości punktu centralnego.  
4. Jeżeli wartość punktu centralnego zmienia się o więcej niż określony próg, algorytm wraca do punktu $2$ i obliczenia są powtarzane. W przeciwnym wypadku algorytm kończy działanie zwracając pogrupowane obiekty. 

Poniżej przedstawiono transformację tablicy dnaych, tak aby mogła zostać przetworzona przez algorytm KNN. W tym celu potrzeba danych w formacie macierzy $m$ x $n$, gdzie $m$ to liczba filmów, a $n$ to liczba użytkowników. Zmiana rozmiaru danych jest możliwa przy wykorzystaniu metody _pivot_. Dodatkowo dla optymalizacji wyników należy przeformatować tablicę danych do macierzy z biblioteki _scipy_. 

In [ ]:
from scipy.sparse import csr_matrix

movie_features = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
mat_movie_features = csr_matrix(movie_features.values)

hashmap = {
            movie: i for i, movie in
            enumerate(list(movies.set_index('movieId').loc[movie_features.index].title))
        }

movie_features

Analizując powyższą macierz, zauważyć można że dane treningowe są bardzo dużych wymiarów. Może być to powodem pogorszenia się wydajności KNN. 

Przykładowo, zastosowanie _odległości Euklidesowej_ w funkcji celu przy dużych wymiarach jest nieprzydatne, ponieważ wszystkie wektory są prawie w równej odległości od wektora głównego. 

Lepszym wyborem miary podobieństwa do odnalezienia najbliższego sąsiada jest _odległość kosinusowa_, która została wykorzystana w poniższej metodzie. _NearestNeighbors_ zostało zaimportowane z biblioteki sklearn, która oferuje wiele algorytmów klasyfikacji, regresji czy klastrowania.

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

<h3>Rekomendacje filmów</h3>

Użytkownik nie musi znać dokładnego tytułu filmu, żeby móc skorzystać z systemu rekomendacyjnego. Dlatego też zaimplementowano metodę _fuzzy_matching_, która zwraca listę filmów, których tytuł jest najbardziej zbliżony do podanego przez użytkownika. 

In [ ]:
from fuzzywuzzy import fuzz

def fuzzy_matching(hashmap, fav_movie):
        match_tuple = []
        for title, idx in hashmap.items():
            ratio = fuzz.ratio(title.lower(), fav_movie.lower())
            if ratio >= 60: 
                match_tuple.append((title, idx, ratio))
        match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
        if not match_tuple:
            print('No match is found')
        else:
            print('Found possible matches in database: {0}\n'.format([x[0] for x in match_tuple]))
            return match_tuple[0][1]

Metoda _inference_, zaimplementowana poniżej, zwraca najlepsze propozycje filmów dla tytułu, podanego przez użytkownika. Do odnalezienia nabliższego sąsiada wykorzystywany jest model uczenia maszynowego. Przy zastosowaniu funkcji, opisanej powyżej, oraz metody _kneighbors_ wytrenowanego modelu, odnajdywane są finalne propozycje filmowe. 

In [ ]:
def inference(model, data, hashmap, fav_movie, n_recommendations):
        model.fit(data)
        print('You have input movie:', fav_movie)
        idx = fuzzy_matching(hashmap, fav_movie)
        print('Recommendation system start to make inference')
        t0 = time.time()
        distances, indices = model.kneighbors(
            data[idx],
            n_neighbors=n_recommendations+1)
        raw_recommends = \
            sorted(
                list(
                    zip(
                        indices.squeeze().tolist(),
                        distances.squeeze().tolist()
                    )
                ),
                key=lambda x: x[1]
            )[:0:-1]
        print('It took {:.2f}s to make inference \n\
              '.format(time.time() - t0))
        return raw_recommends

Z wykorzystaniem metod _fuzzy_matching_ oraz _inference_ możliwa jest implementacja ostatniej funkcji, która posłuży do rekomendacji filmów użytkownikowi, na podstawie podanego przez niego tytułu. Do utworzenia propozycj nie są uwzględniane atrybuty filmu, ani jasno określone preferencje użytkownika. Filmy odnalezione są poprzez zestawienie ze sobą profili preferencji użytkowników z macierzy interakcji oraz zidentyfikowanie zależności.

In [ ]:
def make_recommendations(fav_movie, n_recommendations):
    raw_recommends = inference(model_knn,
                               mat_movie_features, 
                               hashmap, fav_movie, 
                               n_recommendations)
    reverse_hashmap = {v: k for k, v in hashmap.items()}
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance '
              'of {2}'.format(i+1, reverse_hashmap[idx], dist))

In [ ]:
make_recommendations('Iron Man', 10)